<a href="https://colab.research.google.com/github/giacomosansoni/Siamese_Network_Artist_Similarity/blob/main/Triplets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
from tqdm import tqdm
import os
import random

In [ ]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive/My Drive/Thesis/Thesis/

Mounted at /gdrive
/gdrive/.shortcut-targets-by-id/1D4FFhBTYn6QfBuxB5OXjn7JgI3z14YxH/Thesis


In [ ]:
# Finds the all the similar artists given an anchor
def find_positives(anchor, related):
    filtered_df = related[related['musicbrainz_id'] == anchor]
    # Extract the 'id_related_artist' from these filtered rows
    related_artists = filtered_df['id_related_artist'].tolist()
    return related_artists

# Finds a negative artist given an anchor, st it's not in the positives and it's in base_names
def find_negative(anchor, related):
    related_artists = find_positives(anchor, related)
    while True:
      negative = random.choice(base_names)
      if negative not in related_artists and negative in base_names_set:
        break
    return negative

# Add the triplet given anchor and positive
def add_triplet(anchor, positive, negative, triplets, i):
    emb_path = os.path.join(root_dir, 'Emb_artist_scaled', f'{i}', f'{anchor}.npy')
    emb_anc = np.load(emb_path)
    emb_path = os.path.join(root_dir, 'Emb_artist_scaled', f'{i}', f'{negative}.npy')
    emb_neg = np.load(emb_path)
    emb_path = os.path.join(root_dir, 'Emb_artist_scaled', f'{i}', f'{positive}.npy')
    emb_pos = np.load(emb_path)

    assert emb_anc.shape == (1, 61440), f"Error: The shape of emb of {anchor} is {emb_anc.shape}!"
    assert emb_pos.shape == (1, 61440), f"Error: The shape of emb of {positive} is {emb_pos.shape}!"
    assert emb_neg.shape == (1, 61440), f"Error: The shape of emb of {negative} is {emb_pos.shape}!"

    # Concatenate embeddings to form triplets
    triplet = np.concatenate([emb_anc, emb_pos, emb_neg], axis=0)

    triplets = np.concatenate((triplets, triplet[np.newaxis, :]), axis=0)

    return triplets

def add_triplets(anchor, positive, negative, *triplets):
    updated_triplets = []

    for i, triplet_array in enumerate(triplets):
        emb_path = os.path.join(root_dir, 'Emb_artist_scaled', f'{i+1}', f'{anchor}.npy')
        emb_anc = np.load(emb_path)
        emb_path = os.path.join(root_dir, 'Emb_artist_scaled', f'{i+1}', f'{negative}.npy')
        emb_neg = np.load(emb_path)
        emb_path = os.path.join(root_dir, 'Emb_artist_scaled', f'{i+1}', f'{positive}.npy')
        emb_pos = np.load(emb_path)

        assert emb_anc.shape == (1, 61440), f"Error: The shape of emb of {anchor} is {emb_anc.shape}!"
        assert emb_pos.shape == (1, 61440), f"Error: The shape of emb of {positive} is {emb_pos.shape}!"
        assert emb_neg.shape == (1, 61440), f"Error: The shape of emb of {negative} is {emb_neg.shape}!"

        # Concatenate embeddings to form triplets
        triplet = np.concatenate([emb_anc, emb_pos, emb_neg], axis=0)
        triplet_array = np.concatenate((triplet_array, triplet[np.newaxis, :]), axis=0)

        updated_triplets.append(triplet_array)

    return updated_triplets

### Only csv

In [ ]:
# Create an array containing all the musicbrainz_id of the artist we have downloaded
root_dir = os.getcwd()
emb_dir = os.path.join(root_dir, 'Embeddings_40')
base_names_me = []
for file_name in tqdm(os.listdir(emb_dir)):
  if file_name.endswith('.npz'):
    base_name_me = os.path.splitext(file_name)[0]
    base_names_me.append(base_name_me)

# base_names is large, converting it to a set can significantly speed up membership tests
base_names_me_set = set(base_names_me)

# x: Xiangbo, d: Davide
artists_x = pd.read_csv(os.path.join(root_dir, 'Triplets', 'artists_xiangbo.csv'), encoding='ISO-8859-1')
artists_d = pd.read_csv(os.path.join(root_dir, 'Triplets', 'artists_davide.csv'))
base_names_x = artists_x['musicbrainz_id']
base_names_d = artists_d['musicbrainz_id']
base_names_x_set = set(base_names_x)
base_names_d_set = set(base_names_d)

# Find the intersection of all three sets
base_names_set = base_names_me_set & base_names_x_set & base_names_d_set

# Convert the intersection set back to a list for base_names
base_names = list(base_names_set)

100%|██████████| 3992/3992 [00:00<00:00, 208750.41it/s]


In [ ]:
len(base_names)

3008

In [ ]:
triplet_dir = os.path.join(root_dir, 'Triplets')

# Ground truths
related_music = pd.read_csv(os.path.join(root_dir, 'Triplets', 'related-allmusic-artists.csv'))
related_spot = pd.read_csv(os.path.join(root_dir, 'Triplets', 'related-spotify-artists.csv'))

# Initializing the dataframes we will then save
new_rows = []
triplets_ids_music = pd.DataFrame(new_rows, columns=['anchor', 'positive', 'negative'])
triplets_ids_spot = pd.DataFrame(new_rows, columns=['anchor', 'positive', 'negative'])
triplets_ids_music_spot = pd.DataFrame(new_rows, columns=['anchor', 'positive', 'negative'])

# Initializing the pairs set that we need to keep track of the triplets we are adding
existing_pairs_music = set(zip(triplets_ids_music['anchor'], triplets_ids_music['positive']))
existing_pairs_spot = set(zip(triplets_ids_music['anchor'], triplets_ids_music['positive']))

In [ ]:
# ALL MUSIC

count = 0
for index, row in tqdm(related_music.iterrows()):
  anchor = row['musicbrainz_id']
  positive = row['id_related_artist']
  if anchor in base_names_set and positive in base_names_set:
    if (anchor, positive) in existing_pairs_music:
      print(f"{anchor} / {positive} already done")
      continue
    negative = find_negative(anchor, related_music)
    new_row = pd.DataFrame({'anchor': [anchor], 'positive': [positive], 'negative': [negative]})
    triplets_ids_music = pd.concat([triplets_ids_music, new_row], ignore_index=True)
    existing_pairs_music.add((anchor, positive))

    # If 2 artists are similar then also the opposite relation counts! Let's check if there is also the opposite relation in the excel
    condition = (related_music['musicbrainz_id'] == positive) & (related_music['id_related_artist'] == anchor)
    matching_rows = related_music[condition]
    if matching_rows.empty:
      negative = find_negative(positive, related_music)
      new_row = pd.DataFrame({'anchor': [positive], 'positive': [anchor], 'negative': [negative]})
      triplets_ids_music = pd.concat([triplets_ids_music, new_row], ignore_index=True)
      existing_pairs_music.add((positive, anchor))
  else:
    count += 1

4048it [00:56, 127.29it/s]

56cd15a1-0d74-438b-8244-c96ffe1cae03 / 8be0594f-8c13-46bb-ab06-f93ffba5c776 already done


11927it [02:02, 158.79it/s]

b5c4ffa2-82e1-4b72-b7f3-c60afb74b860 / 88a8d8a9-7c9b-4f7b-8700-7f0f7a503688 already done


32848it [03:13, 663.57it/s] 

9f9953f0-68bb-4ce3-aace-2f44c87f0aa3 / 5d0c202c-e30f-40c4-abf5-c0007af0d1cc already done


39641it [03:36, 159.80it/s]

32567eb3-aeca-467f-8b89-a69b11fcb781 / 2a90aa9c-d2ff-49fd-a8e5-2463f657eb45 already done


41961it [03:56, 177.08it/s]


In [ ]:
triplets_ids_music.shape, count

((17673, 3), 29192)

In [ ]:
# Save to a new CSV file
output_path = os.path.join(root_dir, 'Triplets', 'triplets_ids_music.csv')
triplets_ids_music.to_csv(output_path, index=False)

In [ ]:
# SPOTIFY

count = 0
for index, row in tqdm(related_spot.iterrows()):
  anchor = row['musicbrainz_id']
  positive = row['id_related_artist']
  if anchor in base_names_set and positive in base_names_set:
    if (anchor, positive) in existing_pairs_spot:
      print(f"{anchor} / {positive} already done")
      continue
    negative = find_negative(anchor, related_spot)
    new_row = pd.DataFrame({'anchor': [anchor], 'positive': [positive], 'negative': [negative]})
    triplets_ids_spot = pd.concat([triplets_ids_spot, new_row], ignore_index=True)
    existing_pairs_spot.add((anchor, positive))

    # If 2 artists are similar then also the opposite relation counts! Let's check if there is also the opposite relation in the excel
    condition = (related_spot['musicbrainz_id'] == positive) & (related_spot['id_related_artist'] == anchor)
    matching_rows = related_spot[condition]
    if matching_rows.empty:
      negative = find_negative(positive, related_spot)
      new_row = pd.DataFrame({'anchor': [positive], 'positive': [anchor], 'negative': [negative]})
      triplets_ids_spot = pd.concat([triplets_ids_spot, new_row], ignore_index=True)
      existing_pairs_spot.add((positive, anchor))
  else:
    count += 1

# Save to a new CSV file
output_path = os.path.join(root_dir, 'Triplets', 'triplets_ids_spot.csv')
triplets_ids_spot.to_csv(output_path, index=False)

triplets_ids_spot.shape, count

114it [00:03, 36.64it/s]

9ada6fba-df61-4849-9f39-eec9be772b22 / 8f532bd1-c27a-4a7c-80f9-9d163f234623 already done


37343it [04:30, 418.54it/s]

b4265534-cf8f-4a8f-827d-fc490ccd15b8 / ab2241c7-ff64-4b52-90d8-4a7b0517d47c already done


66829it [05:36, 198.82it/s] 


((15764, 3), 56960)

In [ ]:
# ALLMUSIC & SPOTIFY TOGETHER

triplets_ids_music_spot = triplets_ids_music
for index, row in tqdm(triplets_ids_spot.iterrows()):
  anchor = row['anchor']
  positive = row['positive']
  negative = row['negative']
  if (anchor, positive) not in existing_pairs_music:
    new_row = pd.DataFrame({'anchor': [positive], 'positive': [anchor], 'negative': [negative]})
    triplets_ids_music_spot = pd.concat([triplets_ids_music_spot, new_row], ignore_index=True)

output_path = os.path.join(root_dir, 'Triplets', 'triplets_ids_music_spot.csv')
triplets_ids_music_spot.to_csv(output_path, index=False)

triplets_ids_music_spot.shape

15764it [00:11, 1431.76it/s]


(27367, 3)

In [ ]:
del triplets_ids_music_spot
triplets_ids_music_spot = pd.read_csv(os.path.join(root_dir, 'Triplets', 'triplets_ids_music_spot.csv'))
triplets_ids_music_spot.shape

(27367, 3)

In [ ]:
# INTERSECTION BETWEEN ALLMUSIC AND SPOTIFY

triplets_ids_music = pd.read_csv(os.path.join(root_dir, 'Triplets', 'triplets_ids_music.csv'))
triplets_ids_spot = pd.read_csv(os.path.join(root_dir, 'Triplets', 'triplets_ids_spot.csv'))

# Merge the DataFrames on 'anchor' and 'positive' columns
merged_df = pd.merge(triplets_ids_music, triplets_ids_spot[['anchor', 'positive']], on=['anchor', 'positive'], how='inner')
shapee = merged_df.shape

output_path = os.path.join(root_dir, 'Triplets', 'triplets_ids_music_spot_intersection.csv')
merged_df.to_csv(output_path, index=False)

del merged_df

shapee

(6070, 3)

### Emb

In [ ]:
i = 1 #type of embeddings
k = 0 #number of the file
count = 1800
root_dir = os.getcwd()
triplet_dir = os.path.join(root_dir, 'Triplets')
npz_file_path = os.path.join(triplet_dir, 'triplets_prova1_2_2.npz') # npz_file_path = os.path.join(triplet_dir, f'triplets{i}_{k}.npz')

# Load the data and shuffle the rows before selecting the first 3000
triplets_ids_music = pd.read_csv(os.path.join(root_dir, 'Triplets', 'triplets_ids_music.csv')).sample(frac=1).reset_index(drop=True)[:3000]
#triplets_ids_music = pd.read_csv(os.path.join(root_dir, 'Triplets', 'triplets_ids_music.csv'))

# Initialize our triplet
triplets = np.empty((0, 3, 61440), dtype=np.float16)

# Loop over the DataFrame rows
for index, row in tqdm(triplets_ids_music.iterrows()):
  anchor = row['anchor']
  positive = row['positive']
  negative = row['negative']

  triplets = add_triplet(anchor, positive, negative, triplets, i)

'''
  if count % 200 == 0 and index != 0:
    triplets_info.to_csv(os.path.join(triplet_dir, 'triplets.csv'), index=False)
    np.savez_compressed(npz_file_path, triplets=triplets)
    print(triplets.shape)

  if count == 7000:
    print("7000 done!")
    break '''

np.savez_compressed(npz_file_path, triplets=triplets)
triplets.shape

3000it [13:45,  3.63it/s]


(3000, 3, 61440)

In [ ]:
root_dir = os.getcwd()
triplet_dir = os.path.join(root_dir, 'Triplets')

# Load the data and shuffle the rows before selecting the first 3000
triplets_ids_music = pd.read_csv(os.path.join(root_dir, 'Triplets', 'triplets_ids_music.csv')).sample(frac=1).reset_index(drop=True)[:3000]
#triplets_ids_music = pd.read_csv(os.path.join(root_dir, 'Triplets', 'triplets_ids_music.csv'))

# Initialize our triplet
triplets1 = np.empty((0, 3, 61440), dtype=np.float16)
triplets2 = np.empty((0, 3, 61440), dtype=np.float16)
triplets3 = np.empty((0, 3, 61440), dtype=np.float16)
triplets4 = np.empty((0, 3, 61440), dtype=np.float16)

# Loop over the DataFrame rows
for index, row in tqdm(triplets_ids_music.iterrows()):
  anchor = row['anchor']
  positive = row['positive']
  negative = row['negative']

  triplets1, triplets2, triplets3, triplets4 = add_triplets(anchor, positive, negative, triplets1, triplets2, triplets3, triplets4)

triplets1.shape,triplets2.shape, triplets3.shape, triplets4.shape

3000it [57:29,  1.15s/it]


((3000, 3, 61440), (3000, 3, 61440), (3000, 3, 61440), (3000, 3, 61440))

In [ ]:
npz_file_path = os.path.join(triplet_dir, 'triplets_prova1.npz')
np.savez_compressed(npz_file_path, triplets=triplets1)
del triplets1
npz_file_path = os.path.join(triplet_dir, 'triplets_prova2.npz')
np.savez_compressed(npz_file_path, triplets=triplets2)
del triplets2
npz_file_path = os.path.join(triplet_dir, 'triplets_prova3.npz')
np.savez_compressed(npz_file_path, triplets=triplets3)
del triplets3
npz_file_path = os.path.join(triplet_dir, 'triplets_prova4.npz')
np.savez_compressed(npz_file_path, triplets=triplets4)
del triplets4

In [ ]:
triplets.dtype

dtype('float16')

In [ ]:
root_dir = os.getcwd()
path = os.path.join(root_dir, 'Triplets', 'triplets_prova_1_2.npz')
with np.load(path) as data:
  triplets = data['triplets'].astype(np.float32)

root_dir = os.getcwd()
path = os.path.join(root_dir, 'Triplets', 'triplets_prova_1_2.npz')
with np.load(path) as data:
  triplets2 = data['triplets']

triplets[0,0,0], triplets2[0,0,0]

(0.61376953, 0.614)

In [ ]:
emb_path = os.path.join(root_dir, 'Emb_artist_scaled', '1', 'ffe54de6-5cee-4c0f-8329-f5946f9bc3ff.npy')
emb_anc = np.load(emb_path).astype(np.float32)
emb_anc.dtype, emb_anc.shape, emb_anc[0,0]

(dtype('float32'), (1, 61440), 0.4819336)

In [ ]:
emb_path = os.path.join(root_dir, 'Emb_artist_scaled', '1', 'ffe54de6-5cee-4c0f-8329-f5946f9bc3ff.npy')
emb_anc = np.load(emb_path)
emb_anc.dtype, emb_anc.shape, emb_anc[0,0]

(dtype('float16'), (1, 61440), 0.482)

In [ ]:
import sys # triplets_prova_1_2_2

# Assuming 'food' is your DataFrame
size_in_bytes = sys.getsizeof(triplets)
size_in_bytes

1105920144

In [ ]:
import sys # triplets_prova_1_2

# Assuming 'food' is your DataFrame
size_in_bytes = sys.getsizeof(triplets)
size_in_bytes

1105920144

In [ ]:
import sys # triplets_prova_1

# Assuming 'food' is your DataFrame
size_in_bytes = sys.getsizeof(triplets)
size_in_bytes

2211840144

In [ ]:
import sys

# Assuming 'food' is your DataFrame
size_in_bytes = sys.getsizeof(triplets)
size_in_bytes

13029949584

In [ ]:
root_dir = os.getcwd()
path = os.path.join(root_dir, 'Triplets', 'triplets_prova_1.npz')
with np.load(path) as data:
  triplets = data['triplets'].astype(np.float16)
npz_file_path = os.path.join(root_dir, 'Triplets', 'triplets_prova_1_2.npz')
np.savez_compressed(npz_file_path, triplets=triplets)
del triplets

path = os.path.join(root_dir, 'Triplets', 'triplets_prova_2.npz')
with np.load(path) as data:
  triplets = data['triplets'].astype(np.float16)
npz_file_path = os.path.join(root_dir, 'Triplets', 'triplets_prova_2_2.npz')
np.savez_compressed(npz_file_path, triplets=triplets)
del triplets

path = os.path.join(root_dir, 'Triplets', 'triplets_prova_3.npz')
with np.load(path) as data:
  triplets = data['triplets'].astype(np.float16)
npz_file_path = os.path.join(root_dir, 'Triplets', 'triplets_prova_3_2.npz')
np.savez_compressed(npz_file_path, triplets=triplets)
del triplets

path = os.path.join(root_dir, 'Triplets', 'triplets_prova_4.npz')
with np.load(path) as data:
  triplets = data['triplets'].astype(np.float16)
npz_file_path = os.path.join(root_dir, 'Triplets', 'triplets_prova_4_2.npz')
np.savez_compressed(npz_file_path, triplets=triplets)
del triplets

In [ ]:
path = os.path.join(root_dir, 'Triplets', 'triplets_prova_1_2.npz')
with np.load(path) as data:
  triplets = data['triplets']
print(f"{triplets.shape} and {triplets.dtype}")
del triplets

path = os.path.join(root_dir, 'Triplets', 'triplets_prova_2_2.npz')
with np.load(path) as data:
  triplets = data['triplets']
print(f"{triplets.shape} and {triplets.dtype}")
del triplets

path = os.path.join(root_dir, 'Triplets', 'triplets_prova_3_2.npz')
with np.load(path) as data:
  triplets = data['triplets']
print(f"{triplets.shape} and {triplets.dtype}")
del triplets

path = os.path.join(root_dir, 'Triplets', 'triplets_prova_4_2.npz')
with np.load(path) as data:
  triplets = data['triplets']
print(f"{triplets.shape} and {triplets.dtype}")
del triplets

(3000, 3, 61440) and float16
(3000, 3, 61440) and float16
(3000, 3, 61440) and float16
(3000, 3, 61440) and float16


In [ ]:
path = os.path.join(root_dir, 'Triplets', 'triplets_music.npz')
with np.load(path) as data:
  triplets = data['triplets'].astype(np.float16)
npz_file_path = os.path.join(root_dir, 'Triplets', 'triplets_music.npz')
np.savez_compressed(npz_file_path, triplets=triplets)
print(f"{triplets.shape} and {triplets.dtype}")
del triplets

(17673, 3, 61440) and float16
